# Object Speed Estimation

Simple Online RealTime Tracking algorithm (Simpler Version), Kalaman Filter is implemented from scratch. 

Author: Muthu Palaniyappan OL (2020115054), Edumba Vannia Raja (2020115030) 

## Project Dependancies

Run this command to install all dependancies.

```bash
pip install -r requirements.txt
```

Project depends only on [OpenCV](https://opencv.org/) (4.6.0.66), [NumPy](https://numpy.org/) (1.23.2)

## Project Setup

### Seting up in `Linux`

Download these files and save it in same folder.

```sh
wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov7-tiny.cfg
wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov7-tiny.weights
```

### Seting up in `Windows`

Download these files and save it in same folder.

```sh
wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov7-tiny.cfg -o yolov7-tiny.cfg
wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov7-tiny.weights -o yolov7-tiny.weights
```

### Download `Video`

Download [Youtube: Road traffic video for object recognition](https://www.youtube.com/watch?v=wqctLW0Hb_0) video and save it as videoplayback.mp4 in same directory.

In [1]:
# Importing Dependancies
import cv2
import numpy as np
import sort # sort.py

# Neural Network

using pre-trained neural network model yolov7-tiny.

In [2]:
neural_network = cv2.dnn_DetectionModel('yolov7-tiny.weights', 'yolov7-tiny.cfg') # yolov7 pretrained model
neural_network.setInputSize((640, 640)) # input size for yolov7 pretrained model 
neural_network.setInputScale(1.0/255.0) # input range for yolov7 pretrained model
neural_network.setInputSwapRB(True) # opencv's BGR to RGB

< cv2.dnn.Model 0x7f3fcc1c4f30>

In [3]:
def infobox_over_object(frame: np.ndarray, box: list, text: str, box_color: tuple = (255, 100, 40), text_color: tuple = (255, 255, 255)):
    """
    Draws box over and object in the "frame" and puts text above the drawn box over the object.

    frame is opencv frames (numpy).

    box is clockwise starting from left top x_cord , y_cord together. (8 members in list)
    """
    cv2.rectangle(frame, (box[0]-1, box[1]),
                  (box[0]+box[2]+1, box[1]+box[3]), box_color, 2)
    cv2.rectangle(frame, (box[0]-2, box[1] - 18),
                  (box[0]+box[2]+2, box[1]), box_color, -1)
    cv2.putText(frame, text, (box[0], box[1] - 4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1)

In [5]:
cam = cv2.VideoCapture('videoplayback.mp4')
s = sort.SORT(Tlost_max=30, iou_min=0.1)

while True:
    ret_status, frame = cam.read()
    if not ret_status:
        break

    classes, confidences, boxes = neural_network.detect(frame, confThreshold=0.5, nmsThreshold=0.4)

    # Converting top_left edge oriented boxes to center oriented 
    boxes[:, 2] = boxes[:, 2] / np.full(boxes[:, 2].shape, 2.0)
    boxes[:, 3] = boxes[:, 3] / np.full(boxes[:, 3].shape, 2.0)
    boxes[:, 0] += boxes[:, 2]
    boxes[:, 1] += boxes[:, 3]

    # sending it into sort algorithm 
    model_predictions = np.concatenate((boxes, confidences.reshape(-1, 1)), axis=1, dtype=np.float16, casting='unsafe')
    res = s.update(model_predictions)

    # converting center oriented output into topleft edge oriented 
    res[:, 0] = res[:, 0] - res[:, 2]
    res[:, 1] = res[:, 1] - res[:, 3]
    res[:, 6] = res[:, 6] - res[:, 2]
    res[:, 7] = res[:, 7] - res[:, 3]
    res[:, 2] = res[:, 2] * np.full(res[:, 2].shape, 2.0)
    res[:, 3] = res[:, 3] * np.full(res[:, 3].shape, 2.0)
    

    for i in res:
        # Drawing Blue Boxes Which are acctual car detections from yolov7 model
        infobox_over_object(frame, (int(i[0]),int(i[1]),int(i[2]),int(i[3])), str(int(i[5])))
        # Drawing Red Boxes Which are detections predicted by kalaman filter
        infobox_over_object(frame, (int(i[6]), int(i[7]), int(i[2]), int(i[3])), str(int(i[5])), box_color=(0, 0, 255))
    
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1)

    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()